In [1]:
import sys
sys.path.append('C:/Users/320060820/experiments/')
import fxp_quantize
import model_data
import fixed_bitwidth
from model_gen import keras_cnn, inception_cnn
from algorithms.optimized_search import OptimizedSearch

import keras
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# to restart layer numbering
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [2]:
input_shape = (28, 28, 1)
num_outputs = 10 # number of classes
path_trained_weights = 'C:/Users/320060820/experiments/test_models/mnist_custom/mnist_keras_w.h5'
name = 'mnist_keras'

from test_models.mnist_custom import data
_, test_data = data.get_data()

model_arch = keras_cnn.KerasCNN(input_shape, num_outputs, path_trained_weights)
keras_model = model_data.Model(name, test_data, model=model_arch.get_float_model())
float_model_acc = keras_model.evaluate_accuracy()[1]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
input_shape = (32, 32, 3)
num_outputs = 10 # number of classes
path_trained_weights = 'C:/Users/320060820/experiments/test_models/svhn_keras/svhn_keras_w.h5'
name = 'svhn_keras'

from test_models.svhn_keras import data
_, test_data = data.get_data()

model_arch = keras_cnn.KerasCNN(input_shape, num_outputs, path_trained_weights)
model_obj = model_data.Model(name, test_data, model=model_arch.get_float_model())
float_model_acc = model_obj.evaluate_accuracy()[1]

In [11]:
input_shape = (96, 96, 1)
num_outputs = 10 # number of classes
path_trained_weights = 'C:/Users/320060820/experiments/test_models/fashion_df/fashion_df_w.h5'
name = 'fashion_df'

from test_models.fashion_df import data
_, test_data = data.get_data()

model_arch = inception_cnn.InceptionCNN(input_shape, num_outputs, path_trained_weights, dropout=True, 
                                        pool_layer_type='max')
model_obj = model_data.Model(name, test_data, model=model_arch.get_float_model())
float_model_acc = model_obj.evaluate_accuracy()[1]

In [ ]:
# weights
opts = OptimizedSearch(0.003, model_arch, name, test_data, float_model_acc)
scores, opt_params = opts.run('weights', start_bw = 8)
scores_dict = {i: scores[i] for i in scores.keys()}

print(f'Floating point model accuracy: {float_model_acc * 100 :.3f} %')

model_obj = model_data.Model(name, test_data, model=model_arch.get_float_model())
model_obj = fxp_quantize.fix_weights_quantization(model_obj, opt_params)
fxp_model_acc = model_obj.evaluate_accuracy()[1]
print(f'Quantized model accuracy: {fxp_model_acc * 100 :.3f} %')
print(f'Accuracy drop of {(float_model_acc - fxp_model_acc) / float_model_acc * 100 :.3f} %')

opts.plot_scores_matrix(scores_dict, 12, 2, (20, 75))

In [93]:
# biases
opts = OptimizedSearch(0.003, model_arch, name, test_data, float_model_acc)
scores, opt_params = opts.run('biases', start_bw = 8)
scores_dict = {i: scores[i] for i in scores.keys()}

print(f'Floating point model accuracy: {float_model_acc * 100 :.3f} %')

model_obj = model_data.Model(name, test_data, model=model_arch.get_float_model())
model_obj = fxp_quantize.fix_biases_quantization(model_obj, opt_params)
fxp_model_acc = model_obj.evaluate_accuracy()[1]
print(f'Quantized model accuracy: {fxp_model_acc * 100 :.3f} %')
print(f'Accuracy drop of {(float_model_acc - fxp_model_acc) / float_model_acc * 100 :.3f} %')

opts.plot_scores_matrix(scores_dict, 3, 2, (20, 15))

Floating point model accuracy: 0.9911
Quantized model accuracy: 0.9908
Accuracy drop of 0.030269397638983654 %


In [42]:
# activations
opts = OptimizedSearch(0.003, model_arch, name, test_data, float_model_acc)
scores, opt_params = opts.run('activations', start_bw = 8)
scores_dict = {i: scores[i] for i in scores.keys()}

print(f'Floating point model accuracy: {float_model_acc * 100 :.3f} %')

model_obj = model_data.Model(name, test_data, model=model_arch.get_fxp_model(opt_params))
fxp_model_acc = model_obj.evaluate_accuracy()[1]
print(f'Quantized model accuracy: {fxp_model_acc * 100 :.3f} %')
print(f'Accuracy drop of {(float_model_acc - fxp_model_acc) / float_model_acc * 100 :.3f} %')

opts.plot_scores_matrix(scores_dict, 3, 2, (20, 15))

Floating point model accuracy: 0.9911
Quantized model accuracy: 0.9873
Accuracy drop of 0.3834123700938377 %
